In [1]:
import duckdb
import pandas as pd
from pathlib import Path

# Setup

In [2]:
duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,<NA>,6,0.99,183,9.99,G,"Trailers,Behind the Scenes",2021-03-06 15:52:08
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,<NA>,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes",2021-03-06 15:52:08
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,<NA>,6,0.99,105,10.99,NC-17,Deleted Scenes,2021-03-06 15:52:08
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,<NA>,5,2.99,101,28.99,R,"Trailers,Deleted Scenes",2021-03-06 15:52:08


In [4]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description['name']:
        dfs[name] = conn.sql(f"FROM {name};").df()

In [5]:
dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

In [6]:
dfs["actor"].head(3)

,actor_id,first_name,last_name,last_update
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59
2,3.0,ED,CHASE,2021-03-06 15:51:59


In [7]:
for table_name in ["film", "film_actor", "film_category", "actor", "category"]:
    duckdb.register(table_name, dfs[table_name])

duckdb.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",True
1,temp,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",True
2,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
3,temp,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True
4,temp,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True


# Task 1a

# For the tasks, I wanted to have a clean look, but had to add labels like "print()" since it wasn't cooperating

In [73]:
# a) Movies longer than 3 hours (180 minutes)

task1a = duckdb.sql(
    """
    SELECT title, length
    FROM film 
    WHERE length > 180
    ORDER BY length DESC;
"""
).df()

print("=== Task 1a: Movies longer than 3 hours ===")
print(f"Found {len(task1a)} movies")
task1a.head(10)

=== Task 1a: Movies longer than 3 hours ===
Found 39 movies


,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


# Task 1b

In [58]:
# b) Movies with only 'love' in its title

task1b = duckdb.sql(
    """
    SELECT title, rating, length, description
    FROM film
    WHERE REGEXP_MATCHES(title, '\\blove\\b', 'i')
    ORDER BY title;
""").df()

print("=== Task 1b: Movies with only 'love' in its title ===")
print(f"Found {len(task1b)} movies")
task1b

=== Task 1b: Movies with only 'love' in its title ===
Found 5 movies


,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


In [66]:
# c) Descriptive statistics on movie length column

task1c = duckdb.sql(
    """
    SELECT 
        MIN(length) AS shortest,
        AVG(length) AS average,
        Percentile_Cont(0.0) WITHIN GROUP (ORDER BY length) AS median,
        MAX(length) AS longest
    FROM film;
""").df()

print("=== Task 1c: Descriptive statistics on movie length column ===")
task1c

=== Task 1c: Descriptive statistics on movie length column ===


,shortest,average,median,longest
0,46,115.272,46.0,185


In [68]:
# d) 10 most expensive movies to rent per day

task1d = duckdb.sql(
    """
    SELECT
        title, 
        rental_rate,
        rental_duration,
        Round(rental_rate / rental_duration, 2) AS cost_per_day
    FROM film
    ORDER BY cost_per_day DESC
    LIMIT 10;
""").df()

print("=== Task 1d: 10 most expensive movies to rent per day ===")
task1d

=== Task 1d: 10 most expensive movies to rent per day ===


,title,rental_rate,rental_duration,cost_per_day
0,AMERICAN CIRCUS,4.99,3,1.66
1,BACKLASH UNDEFEATED,4.99,3,1.66
2,BILKO ANONYMOUS,4.99,3,1.66
3,BEAST HUNCHBACK,4.99,3,1.66
4,CARIBBEAN LIBERTY,4.99,3,1.66
5,AUTUMN CROW,4.99,3,1.66
6,CASPER DRAGONFLY,4.99,3,1.66
7,ACE GOLDFINGER,4.99,3,1.66
8,BEHAVIOR RUNAWAY,4.99,3,1.66
9,CASUALTIES ENCINO,4.99,3,1.66


In [8]:
# e) Actors who played in most movies

task1e = duckdb.sql(
    """
    SELECT 
        a.first_name || ' ' || a.last_name AS actor_name,
        COUNT(fa.film_id) AS movie_count
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
    ORDER BY movie_count DESC
    LIMIT 10;
""").df()

print("=== Task 1e: Actors who played in most movies ===")
task1e

=== Task 1e: Actors who played in most movies ===


,actor_name,movie_count
0,GINA DEGENERES,42
1,WALTER TORN,41
2,MARY KEITEL,40
3,MATTHEW CARREY,39
4,SANDRA KILMER,37
5,SCARLETT DAMON,36
6,VIVIEN BASINGER,35
7,GROUCHO DUNST,35
8,HENRY BERRY,35
9,ANGELA WITHERSPOON,35


In [13]:
print("\n" + "-"*40)

# f) 3-5 of my own questions

print("Question 1: What are the most popular film categories?")

q1 = duckdb.sql(
    """
    SELECT
        c.name as category, 
        COUNT(fc.film_id) AS film_count
    FROM category c
    JOIN film_category fc ON c.category_id = fc.category_id
    GROUP BY c.category_id, c.name
    ORDER by film_count DESC;
""").df()

q1.head()


----------------------------------------
Question 1: What are the most popular film categories?


,category,film_count
0,Sports,74
1,Foreign,73
2,Family,69
3,Documentary,68
4,Animation,66


In [80]:
print("\nQuestion 2: How many movies per rating?")

q2 = duckdb.sql(
    """
    SELECT
        rating,
        COUNT(*) as movie_count,
        AVG(length) as avg_length,
        AVG(rental_rate) as avg_rental_rate
    FROM film
    GROUP BY rating
    ORDER BY movie_count DESC;
""").df()

q2


Question 2: How many movies per rating?


,rating,movie_count,avg_length,avg_rental_rate
0,PG-13,223,120.443946,3.034843
1,NC-17,210,113.228571,2.970952
2,R,195,118.661538,2.938718
3,PG,194,112.005155,3.051856
4,G,178,111.050562,2.888876


In [ ]:
# Some questions didn't work, since not all tables were seemingly transferred
# With the help of LLMs, this test answered that question for me

all_tables = duckdb.sql("SHOW TABLES").df()
print("All tables in database:")
all_tables

table_list = all_tables['name'].tolist()
print("Table names:")
for table in table_list:
    print(f"  - {table}")

# Now time for new questions

All tables in database:
Table names:
  - actor
  - category
  - film
  - film_actor
  - film_category


In [27]:
print("\nQuestion 3: Which actors appear in the most diverse categories?")

q3 = duckdb.sql(
    """
    SELECT 
        a.first_name || ' ' || a.last_name as actor_name,
        COUNT(DISTINCT c.name) as unique_categories,
        COUNT(fa.film_id) as total_movies
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    JOIN film_category fc ON fa.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    GROUP BY a.actor_id, a.first_name, a.last_name
    ORDER BY unique_categories DESC, total_movies DESC
    LIMIT 10;
""").df()

q3


Question 3: Which actors appear in the most diverse categories?


,actor_name,unique_categories,total_movies
0,GINA DEGENERES,16,42
1,UMA WOOD,16,35
2,GROUCHO DUNST,16,35
3,KEVIN GARLAND,16,33
4,REESE WEST,16,33
5,EWAN GOODING,16,33
6,HARVEY HOPE,16,32
7,DARYL WAHLBERG,16,31
8,IAN TANDY,16,31
9,MICHAEL BOLGER,16,30


In [28]:
print("\nQuestion 4: Which actor pairs appear together in most films?")

q4 = duckdb.sql(
    """
    SELECT
        a1.first_name || ' ' || a1.last_name as actor1,
        a2.first_name || ' ' || a2.last_name as actor2,
        COUNT(DISTINCT f.film_id) as films_together
    FROM film_actor fa1
    JOIN film_actor fa2 ON fa1.film_id = fa2.film_id AND fa1.actor_id < fa2.actor_id
    JOIN actor a1 ON fa1.actor_id = a1.actor_id
    JOIN actor a2 ON fa2.actor_id = a2.actor_id
    JOIN film f ON fa1.film_id = f.film_id
    GROUP BY a1.actor_id, a1.first_name, a1.last_name, 
             a2.actor_id, a2.first_name, a2.last_name
    ORDER BY films_together DESC
    LIMIT 10;
""").df()

q4



Question 4: Which actor pairs appear together in most films?


,actor1,actor2,films_together
0,JULIA MCQUEEN,HENRY BERRY,7
1,VAL BOLGER,ROCK DUKAKIS,6
2,BEN WILLIS,HARVEY HOPE,6
3,CUBA OLIVIER,MARY KEITEL,6
4,KIRSTEN PALTROW,WARREN NOLTE,6
5,MORGAN MCDORMAND,WILL WILSON,6
6,JOHNNY LOLLOBRIGIDA,MICHAEL BOLGER,5
7,MERYL GIBSON,ED GUINESS,5
8,SEAN GUINESS,MARY KEITEL,5
9,WALTER TORN,EWAN GOODING,5
